Importar librerias

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

definir la URL y realizar conexion esperando respuesta con estatus 200

In [3]:
url='https://www.eltiempo.com/'
tiempo=requests.get(url)
tiempo.status_code

200

Crear sopa de letras con la url

In [4]:
tiempo_soup=BeautifulSoup(tiempo.text,'lxml')
type(tiempo_soup)

bs4.BeautifulSoup

Creando lista con secciones

In [23]:
secciones=tiempo_soup.find('ul',attrs={'class':'default-menu'}).find_all('li')

# Usando list comprehension
#lista_notas = [url + article.a.get('href') for article in articles.find_all('div', attrs={'class': 'article-item__content'}) if article.a]

#Lista Secciones
nombre_secciones=[nseccion.a.get_text('href') for nseccion in secciones]
link_secciones=[url + lseccion.a.get('href') for lseccion in secciones]

In [24]:
nombre_secciones

['Opinión',
 'Colombia',
 'Bogotá',
 'Internacional',
 'Política',
 'Justicia',
 'Economía',
 'Deportes',
 'Cultura',
 'Tecnología',
 'Vida',
 'Unidad Investigativa',
 'Salud']

In [25]:
link_secciones

['https://www.eltiempo.com//opinion',
 'https://www.eltiempo.com//colombia',
 'https://www.eltiempo.com//bogota',
 'https://www.eltiempo.com//mundo',
 'https://www.eltiempo.com//politica',
 'https://www.eltiempo.com//justicia',
 'https://www.eltiempo.com//economia',
 'https://www.eltiempo.com//deportes',
 'https://www.eltiempo.com//cultura',
 'https://www.eltiempo.com//tecnosfera',
 'https://www.eltiempo.com//vida',
 'https://www.eltiempo.com//unidad-investigativa',
 'https://www.eltiempo.com//salud']

In [32]:
secciones_data = {'nombre_seccion': nombre_secciones, 'Enlace': link_secciones}
secciones_df = pd.DataFrame(secciones_data)
#secciones_df = secciones_df[~secciones_df['nombre_seccion'].str.contains('opinión', case=False)]

Columnistas de opinion por dia

In [129]:
url_dia_columns=('https://www.eltiempo.com//colombia')
dia_columns=requests.get(url_dia_columns)
dia_columns.status_code

tcolombia_soup=BeautifulSoup(dia_columns.text,'lxml')

In [284]:
noticias = []
# Encuentra todos los elementos 'article' con la clase 'simple-image'
articles = tcolombia_soup.find_all('article', class_='simple-image')

articles

[<article class="simple-image" data-autor="Melissa munera zambrano" data-bloqueo="Abierto" data-board="Hs : Colombia" data-category="Colombia/santander" data-clasecontenido="Estandar" data-editorial="Editorial" data-id="795610" data-name="Rodolfo hernandez no se ha sometido a ninguna cirugia para extraer su tumor" data-publicacion="2023-08-12" data-redactorvisible="Redaccion el tiempo" data-seccion="Colombia" data-subseccion="Santander" data-tipocontenido="Articulo" id="m1093-2-1094" itemscope="" itemtype="https://schema.org/NewsArticle"><meta itemid="https://www.eltiempo.com/colombia/santander/rodolfo-hernandez-desmiente-cirugia-para-sacar-su-tumor-795610" itemprop="mainEntityOfPage" itemscope="" itemtype="https://schema.org/WebPage"/><div class="recurso" itemtype="https://schema.org/NewsArticle">
 <a class="multimediatag page-link" href="/colombia/santander/rodolfo-hernandez-desmiente-cirugia-para-sacar-su-tumor-795610">
 </a>
 <figure class="foto-desktop-noticias-seccion"><a class="

In [283]:
noticias = []

# Encuentra todos los elementos 'article' con la clase 'simple-image'
articles = tcolombia_soup.find_all('article', class_='simple-image')

for article in articles:
    # Encuentra el elemento 'a' dentro del 'article' con la clase 'boton'
    link = article.find('a', class_='boton')
    if link:
        # Extrae el texto y el atributo 'href' del enlace
        nombre = link.get_text()
        enlace = link['href']
        noticias.append({'nombre': nombre, 'enlace': enlace})

# Imprime la lista de noticias
for noticia in noticias:
    print(noticia)

{'nombre': 'Rodolfo Hernández no se ha sometido a cirugía para extraer su tumor', 'enlace': '/colombia/santander/rodolfo-hernandez-desmiente-cirugia-para-sacar-su-tumor-795610'}
{'nombre': 'Estos son los 310 candidatos al Concejo de Medellín', 'enlace': '/colombia/medellin/medellin-estos-son-los-310-candidatos-al-concejo-en-las-17-listas-inscritas-795358'}
{'nombre': 'Estos son los candidatos a Alcaldía de Floridablanca', 'enlace': '/colombia/santander/floridablanca-estos-son-los-candidatos-a-la-alcaldia-y-algunos-estan-investigados-795244'}
{'nombre': 'Elecciones 2023: los candidatos que van por la Alcaldía de Medellín', 'enlace': '/colombia/medellin/elecciones-medellin-2023-los-candidatos-inscritos-en-puja-por-la-alcaldia-790990'}
{'nombre': 'Cartagena tiene 15 candidatos inscritos a la Alcaldía', 'enlace': '/colombia/otras-ciudades/cartagena-candidatos-inscritos-a-la-alcaldia-791402'}
{'nombre': 'Elecciones 2023: los candidatos que aspiran a la Alcaldía de Neiva', 'enlace': '/colomb

In [274]:
nombre_titulos

['', '', '', '']

In [83]:
link_titulos

['https://www.eltiempo.com//colombia/cali/dilian-francisca-toro-candidata-a-la-gobernacion-del-valle-2023-796507']

In [104]:
titulos=[]
titulos=tcolombia_soup.find('a',attrs={'class':'multimediatag page-link'})

#Lista Secciones
nombre_titulos=[ntitulos.get_text() for ntitulos in titulos]
link_titulos=[url + ltitulos.get('href') for ltitulos in titulos]

AttributeError: 'NavigableString' object has no attribute 'get'

In [101]:
nombre_titulos

['',
 'Santander',
 '\n',
 'Estos son los candidatos a Alcaldía de Floridablanca',
 '',
 'Medellín',
 '\n',
 'Elecciones 2023: los candidatos que van por la Alcaldía de Medellín',
 '',
 'Otras Ciudades',
 '\n',
 'Cartagena tiene 15 candidatos inscritos a la Alcaldía',
 '',
 'Otras Ciudades',
 '\n',
 'Elecciones 2023: los candidatos que aspiran a la Alcaldía de Neiva']